In [1]:
# Make sure local Python modules in "Contrib" can be imported
import sys
import os

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Set current directory to where data is
import configparser
config = configparser.ConfigParser()
config.read('jupyter_satellites.config')
data_path = config.get('Jupyter_Notebooks', 'data_path')
os.chdir(data_path)

In [9]:
import ipywidgets as widgets

# Local module for geocoding
import Satellite.Contrib.addressGeocode as geocode

address=widgets.Text(
    description='Address',
    layout=widgets.Layout(width='100%')
)

# Global variable to store desired location
latlong = 0

def handle_geocode_submit(sender):
    global latlong
    latlong = geocode.addressGeocode(address.value)
    print(latlong)
    
address.on_submit(handle_geocode_submit)

display(address)

Text(value='', description='Address', layout=Layout(width='100%'))

{'x': 145.104084, 'y': -38.198223}


In [8]:
print(latlong)

{'x': 145.104084, 'y': -38.198223}
